## LOTR: Elven Wildcards

[Source: CodeWars](https://www.codewars.com/kata/5ad90fb688a0b74111000055/train/sql)

Deep within the fair realm of Lothlórien, you have been asked to create a shortlist of candidates for a recently vacated position on the council.

Of so many worthy elves, who to choose for such a lofty position? After much thought you decide to select candidates by name, which are often closely aligned to an elf's skill and temperament.

Choose those with "tegil" appearing anywhere in their first name, as they are likely to be good calligraphers, OR those with "astar" anywhere in their last name, who will be faithful to the role.

Table Overview: `elves`

- firstname
- lastname

Note: all names are in lowercase.

To aid the scribes, return the `firstname` and `lastname` column concatenated, separated by a space, into a single `shortlist` column, and capitalise the first letter of each name.

<img src="LOTR-elf-names.png" alt="elves table screenshot, from codewars" width=500>

In [ ]:
'''
--but on the land of Lórien no shadow lay--

SELECT concat(INITCAP(firstname), ' ', INITCAP(lastname)) AS shortlist
FROM elves
WHERE firstname LIKE '%tegil%'
OR lastname LIKE '%astar%';
'''

In [ ]:
# another solution option:
'''
SELECT INITCAP(firstname || ' ' || lastname) AS shortlist
FROM elves
WHERE firstname LIKE '%tegil%' 
OR lastname LIKE '%astar%'
'''

-----

## Harry Potter: Sorting Hat

[Source: CodeWars](https://www.codewars.com/kata/5abcf0f930488ff1a6000b66/train/sql)

There is truly no magic in the world; the Hogwarts Sorting Hat is SQL-based, its decision-making powers are common operators and prospective students are merely data - names, and two columns of qualities.

Table Overview: `students`

- id
- name
- quality1
- quality2

Conditions: 

- Slytherin are being quite strict this year and will only take students who are 'evil' AND 'cunning'

- Gryffindor will take students who are 'brave' but only if their second quality is NOT 'evil'

- Ravenclaw accepts students who are 'studious' OR 'intelligent'

- Hufflepuff will simply take those who have the quality 'hufflepuff'

(don't worry, for simplicity's sake 'brave' and 'studious' will only appear in `quality1`, and 'cunning' and 'intelligent' will only appear in `quality2`.)

Return the `id`, `name`, `quality1` and `quality2` of all the students who'll be accepted, ordered by ascending `id`.

<img src="HP-sorting-hat.png" alt="sorting hat table screenshot, from codewars" width=500>

In [ ]:
'''
/* Oh you may not think I'm pretty,
But don't judge on what you see,
I'll eat myself if you can find
A smarter hat than me. */

SELECT * 
FROM students
WHERE (quality1 = 'evil' AND quality2 = 'cunning' )
OR (quality1 = 'brave' AND quality2 != 'evil')
OR (quality1 = 'studious' OR quality2 = 'intelligent')
OR (quality1 = 'hufflepuff' OR quality2 = 'hufflepuff')
ORDER BY id
'''

In [ ]:
# another solution

'''

WITH
    sorted_students AS (
      SELECT
        s.*,
        CASE
        WHEN (s.quality1 = 'evil' AND s.quality2 = 'cunning')
             OR (s.quality1 = 'cunning' AND s.quality2 = 'evil')
          THEN 'Slytherin'
        WHEN (s.quality1 = 'brave' AND s.quality2 != 'evil')
             OR (s.quality1 != 'evil' AND s.quality2 = 'brave')
          THEN 'Gryffindor'
        WHEN (s.quality1 = 'studious' OR s.quality2 = 'studious' OR s.quality1 = 'intelligent' OR
              s.quality2 = 'intelligent')
          THEN 'Ravenclaw'
        WHEN (s.quality1 = 'hufflepuff' OR s.quality2 = 'hufflepuff')
          THEN 'Hufflepuff'
        ELSE NULL
        END AS house
      FROM students s
  )
SELECT
  ss.id,
  ss.name,
  ss.quality1,
  ss.quality2
FROM sorted_students ss
WHERE ss.house NOTNULL
ORDER BY ss.id;
'''

-----

## LEVEL UP!

## Actors Cast Together

[Source: CodeWars](https://www.codewars.com/kata/5818bde9559ff58bd90004a2)

Given the the schema presented below find two actors who are cast together the most, and list titles of only those movies they were cast in together. Order the result set alphabetically by the movie title.

Table Overview: `film_actor`
- actor_id
- film_id

Table Overview: `actor`
- actor_id
- first_name
- last_name

Table Overview: `film`
- film_id
- title


The desired output:

first_actor | second_actor | title
------------|--------------|--------------------
John Doe    | Jane Doe     | The Best Movie Ever

- `first_actor` - Full name (First name + Last name separated by a space)
- `second_actor` - Full name (First name + Last name separated by a space)
- `title` - Movie title

Note: `actor_id` of the first_actor should be lower than `actor_id` of the second_actor

`film_actor` Table:
<img src="Movies-film-actor.png" alt="film/actor table screenshot, from codewars" width=500>

`actor` Table:
<img src="Movies-actor.png" alt="actor table screenshot, from codewars" width=500>

`film` Table:
<img src="Movies-film.png" alt="film table screenshot, from codewars" width=500>


In [ ]:
'''
SELECT 
  CONCAT(aa.first_name, ' ', aa.last_name) AS first_actor, 
  CONCAT(ab.first_name, ' ', ab.last_name) AS second_actor, 
  fi.title
  
-- First subquery: finding top combo
FROM (
  SELECT 
    faa.actor_id AS first_id, 
    fab.actor_id AS second_id, 
    COUNT(*) AS tot
  FROM film_actor faa
  INNER JOIN film_actor fab 
  ON faa.film_id = fab.film_id
  AND faa.actor_id < fab.actor_id
  GROUP BY faa.actor_id, fab.actor_id
  ORDER BY tot DESC
  LIMIT 1
  ) AS top_combo
  
-- Second subquery: grabbing rows from the top combo
JOIN (
  SELECT 
    faa.actor_id AS first_id, 
    fab.actor_id AS second_id,
    faa.film_id AS film_id
  FROM film_actor faa
  INNER JOIN film_actor fab 
  ON faa.film_id = fab.film_id
  AND faa.actor_id < fab.actor_id
) AS combos

ON top_combo.first_id = combos.first_id
AND top_combo.second_id = combos.second_id

-- Now grabbing first actor details
JOIN actor aa
ON combos.first_id = aa.actor_id
-- Grabbing second actor details
JOIN actor ab
ON combos.second_id = ab.actor_id
-- And grabbing film details
JOIN film fi
ON combos.film_id = fi.film_id;

'''

In [ ]:
'''
WITH top_pair AS (
  SELECT a1.actor_id AS id1, a2.actor_id AS id2
  FROM film_actor a1
    INNER JOIN film_actor a2 ON a1.film_id=a2.film_id
  WHERE a1.actor_id <> a2.actor_id
  GROUP BY a1.actor_id, a2.actor_id
  ORDER BY COUNT(a1.film_id) DESC
  LIMIT 1
)
SELECT
(SELECT first_name || ' ' || last_name FROM actor WHERE actor_id = tp.id1) AS first_actor,
(SELECT first_name || ' ' || last_name FROM actor WHERE actor_id = tp.id2) AS second_actor,
  f.title AS title
FROM top_pair tp
    INNER JOIN film_actor fa1 ON tp.id1 = fa1.actor_id
    INNER JOIN film_actor fa2 ON tp.id2 = fa2.actor_id
    INNER JOIN film f ON fa1.film_id=f.film_id AND fa2.film_id=f.film_id
'''